Imports

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import random
import re
import glob
from PIL import Image,ImageDraw,ImageFont

Load annotated font .csv and define root directory containing google fonts repo

In [ ]:
df = pd.read_csv('google-fonts-annotaion.csv')

# change your path here
rds_dir = '/rds-d2/user/wjm41/hpc-work/datasets'
ROOT = rds_dir+'/fonts/ofl'

define font filtering & retrieval function

In [ ]:
def filter_fonts_get_paths(df, root='./', variants=['_'], subsets=['_'], category=''):
    # exceptions
    if not variants or variants == [''] or variants == '': variants = ['_']
    if not subsets or subsets == [''] or subsets == '': subsets = ['_']
    # apply filters
    regex_filters = variants + subsets + ['_'+category]
    df_new = pd.concat([df.filter(regex=re.compile(regex, re.IGNORECASE), axis=1).sum(axis=1).astype(bool) for regex in regex_filters], axis=1)
    mask = df_new.all(axis=1)
    filtered_fontnames = list(df.loc[mask].family)
    # construct file paths
    paths = []
    for fontname in filtered_fontnames:
        if variants == ['_']: # select all variants
            sel = glob.glob(f'{root}/{fontname.lower()}/**/**/**/*.ttf', recursive=True)
            paths.extend(sel)
        else:
            for variant in variants:
                sel = glob.glob(f'{root}/{fontname.lower()}/**/**/{fontname}-{variant}.ttf', recursive=True) 
                for path in sel:
                    paths.append(path)
    print(f'Found {len(paths)} font files.')
    return paths

Draw text example using a random font!

In [ ]:
paths = filter_fonts_get_paths(df, root=ROOT, subsets=['chinese-simplified'], variants=['_'], category='')
r = random.randrange(0, len(paths))

# sample text and font
text = "新年快樂"
text = "身體健康"
text = "心想事成"

text_size = 150
font = ImageFont.truetype(paths[r], text_size)

# # get text info (not being used but may be useful)
text_width, text_height = font.getsize(text)
left, top, right, bottom = font.getbbox(text)
print('text w & h: ', text_width, text_height)
print(left, top, right, bottom)

IMG_WIDTH = 700
IMG_HEIGHT = 200

x = 300 # we can adjust this according to IMG_WIDTH etc
y = 150

# create a blank canvas with extra space between lines
canvas = Image.new('RGB', (IMG_WIDTH, IMG_HEIGHT), "red")

# draw the text onto the text canvas
draw = ImageDraw.Draw(canvas)
draw.text((x, y), text, 'black', font, anchor='ms')

fig = plt.figure(figsize=(30,10))
plt.imshow(canvas, aspect='auto')